In [1]:
import sqlite3
import pandas as pd
import os
import sqlalchemy.types

In [2]:
%load_ext sql

In [3]:
# check the data files
os.listdir()

['.ipynb_checkpoints',
 '01. Composing and Running basic SQL queries.ipynb',
 '02. String Patterns, Sorting & Grouping.ipynb',
 '03. Built-in functions.ipynb',
 '04. Sub-Queries and Nested-Selects.ipynb',
 '05. JOIN Operations.ipynb',
 'CREATE and DROP tables.pdf',
 'Create Database Instance.pdf',
 'database.db',
 'Departments.csv',
 'Employees.csv',
 'HR.db',
 'Jobs.csv',
 'JobsHistory.csv',
 'Locations.csv',
 'Pet.db',
 'README.md']

In [4]:
df_Dep = pd.read_csv('Departments.csv', header=None)
df_Dep.rename(columns = {0:'DEPT_ID_DEP', 1:'DEP_NAME', 2:'MANAGER_ID', 3:'LOC_ID'}, inplace=True)
df_Dep

,DEPT_ID_DEP,DEP_NAME,MANAGER_ID,LOC_ID
0,2,Architect Group,30001,L0001
1,5,Software Group,30002,L0002
2,7,Design Team,30003,L0003
3,5,Software Group,30004,L0004


In [5]:
df_Emp = pd.read_csv('Employees.csv', header=None)
df_Emp.rename(columns = {0:'EMP_ID', 1:'F_NAME', 2:'L_NAME', 3:'SSN', 4:'B_DATE', 5:'SEX',
                        6:'ADDRESS', 7:'JOB_ID', 8:'SALARY', 9:'MANAGER_ID', 10:'DEP_ID'}, inplace=True)
df_Emp['B_DATE'] = pd.to_datetime(df_Emp["B_DATE"]).dt.strftime('%Y-%m-%d')
df_Emp

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1001,John,Thomas,123456,1976-01-09,M,"5631 Rice, OakPark,IL",100,100000,30001,2
1,E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
2,E1003,Steve,Wells,123458,1980-08-10,M,"291 Springs, Gary,IL",300,50000,30002,5
3,E1004,Santosh,Kumar,123459,1985-07-20,M,"511 Aurora Av, Aurora,IL",400,60000,30004,5
4,E1005,Ahmed,Hussain,123410,1981-01-04,M,"216 Oak Tree, Geneva,IL",500,70000,30001,2
5,E1006,Nancy,Allen,123411,1978-02-06,F,"111 Green Pl, Elgin,IL",600,90000,30001,2
6,E1007,Mary,Thomas,123412,1975-05-05,F,"100 Rose Pl, Gary,IL",650,65000,30003,7
7,E1008,Bharath,Gupta,123413,1985-05-06,M,"145 Berry Ln, Naperville,IL",660,65000,30003,7
8,E1009,Andrea,Jones,123414,1990-07-09,F,"120 Fall Creek, Gary,IL",234,70000,30003,7
9,E1010,Ann,Jacob,123415,1982-03-30,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5


In [6]:
df_Job = pd.read_csv('Jobs.csv', header=None)
df_Job.rename(columns = {0:'JOB_IDENT', 1:'JOB_TITLE', 2:'MIN_SALARY', 3:'MAX_SALARY'}, inplace=True)
df_Job

,JOB_IDENT,JOB_TITLE,MIN_SALARY,MAX_SALARY
0,100,Sr. Architect,60000,100000
1,200,Sr. Software Developer,60000,80000
2,300,Jr.Software Developer,40000,60000
3,400,Jr.Software Developer,40000,60000
4,500,Jr. Architect,50000,70000
5,600,Lead Architect,70000,100000
6,650,Jr. Designer,60000,70000
7,660,Jr. Designer,60000,70000
8,234,Sr. Designer,70000,90000
9,220,Sr. Designer,70000,90000


In [7]:
df_JobHis = pd.read_csv('JobsHistory.csv', header=None)
df_JobHis.rename(columns = {0:'EMPL_ID', 1:'START_DATE', 2:'JOBS_ID', 3:'DEPT_ID'}, inplace=True)
df_JobHis

,EMPL_ID,START_DATE,JOBS_ID,DEPT_ID
0,E1001,08/01/2000,100,2
1,E1002,08/01/2001,200,5
2,E1003,08/16/2001,300,5
3,E1004,08/16/2000,400,5
4,E1005,05/30/2000,500,2
5,E1006,08/16/2001,600,2
6,E1007,05/30/2002,650,7
7,E1008,05/06/2010,660,7
8,E1009,08/16/2016,234,7
9,E1010,08/16/2016,220,5


In [8]:
df_Loc = pd.read_csv('Locations.csv', header=None)
df_Loc.rename(columns = {0:'LOCT_ID', 1:'DEP_ID_LOC'}, inplace=True)
df_Loc

,LOCT_ID,DEP_ID_LOC
0,L0001,2
1,L0002,5
2,L0003,7


### Setup database

In [9]:
# Create 'HR' database

conn = sqlite3.connect("HR.db")

# create ALL tables
df_Dep.to_sql("Departments", conn, if_exists='replace', index=False, method="multi")
df_Emp.to_sql("Employees", conn, if_exists='replace', index=False, method="multi", dtype={'B_DATE':'Date'})
df_Job.to_sql("Jobs", conn, if_exists='replace', index=False, method="multi")
df_JobHis.to_sql("JobsHistory", conn, if_exists='replace', index=False, method="multi")
df_Loc.to_sql("Locations", conn, if_exists='replace', index=False, method="multi")

3

In [10]:
%sql sqlite:///HR.db

### Query 1: Retrieve all employees whose address is in Elgin,IL

In [11]:
%%sql 

SELECT * from Employees
    WHERE ADDRESS LIKE '%Elgin,IL'

 * sqlite:///HR.db
Done.


EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
E1006,Nancy,Allen,123411,1978-02-06,F,"111 Green Pl, Elgin,IL",600,90000,30001,2
E1010,Ann,Jacob,123415,1982-03-30,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5


### Query 2: Retrieve all employees who were born during the 1970's

In [12]:
%%sql

SELECT * from Employees
    WHERE B_DATE LIKE '%197%'

 * sqlite:///HR.db
Done.


EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
E1001,John,Thomas,123456,1976-01-09,M,"5631 Rice, OakPark,IL",100,100000,30001,2
E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
E1006,Nancy,Allen,123411,1978-02-06,F,"111 Green Pl, Elgin,IL",600,90000,30001,2
E1007,Mary,Thomas,123412,1975-05-05,F,"100 Rose Pl, Gary,IL",650,65000,30003,7


### Query 3: Retrieve all employees in department 5 whose salary is between 60000 and 70000 .

In [13]:
%%sql 

SELECT * from Employees
    WHERE (SALARY BETWEEN 60000 and 70000) and DEP_ID = 5

 * sqlite:///HR.db
Done.


EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
E1004,Santosh,Kumar,123459,1985-07-20,M,"511 Aurora Av, Aurora,IL",400,60000,30004,5
E1010,Ann,Jacob,123415,1982-03-30,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5


### Query 4A: Retrieve a list of employees ordered by department ID. 

In [14]:
%%sql

SELECT * from Employees
    ORDER BY DEP_ID 

 * sqlite:///HR.db
Done.


EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
E1001,John,Thomas,123456,1976-01-09,M,"5631 Rice, OakPark,IL",100,100000,30001,2
E1005,Ahmed,Hussain,123410,1981-01-04,M,"216 Oak Tree, Geneva,IL",500,70000,30001,2
E1006,Nancy,Allen,123411,1978-02-06,F,"111 Green Pl, Elgin,IL",600,90000,30001,2
E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
E1003,Steve,Wells,123458,1980-08-10,M,"291 Springs, Gary,IL",300,50000,30002,5
E1004,Santosh,Kumar,123459,1985-07-20,M,"511 Aurora Av, Aurora,IL",400,60000,30004,5
E1010,Ann,Jacob,123415,1982-03-30,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5
E1007,Mary,Thomas,123412,1975-05-05,F,"100 Rose Pl, Gary,IL",650,65000,30003,7
E1008,Bharath,Gupta,123413,1985-05-06,M,"145 Berry Ln, Naperville,IL",660,65000,30003,7
E1009,Andrea,Jones,123414,1990-07-09,F,"120 Fall Creek, Gary,IL",234,70000,30003,7


### Query 4B: Retrieve a list of employees ordered in descending order by department ID and within each department ordered alphabetically in descending order by last name.

In [15]:
%%sql

SELECT * from Employees
    ORDER BY DEP_ID DESC, L_NAME DESC

 * sqlite:///HR.db
Done.


EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
E1007,Mary,Thomas,123412,1975-05-05,F,"100 Rose Pl, Gary,IL",650,65000,30003,7
E1009,Andrea,Jones,123414,1990-07-09,F,"120 Fall Creek, Gary,IL",234,70000,30003,7
E1008,Bharath,Gupta,123413,1985-05-06,M,"145 Berry Ln, Naperville,IL",660,65000,30003,7
E1003,Steve,Wells,123458,1980-08-10,M,"291 Springs, Gary,IL",300,50000,30002,5
E1004,Santosh,Kumar,123459,1985-07-20,M,"511 Aurora Av, Aurora,IL",400,60000,30004,5
E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000,30002,5
E1010,Ann,Jacob,123415,1982-03-30,F,"111 Britany Springs,Elgin,IL",220,70000,30004,5
E1001,John,Thomas,123456,1976-01-09,M,"5631 Rice, OakPark,IL",100,100000,30001,2
E1005,Ahmed,Hussain,123410,1981-01-04,M,"216 Oak Tree, Geneva,IL",500,70000,30001,2
E1006,Nancy,Allen,123411,1978-02-06,F,"111 Green Pl, Elgin,IL",600,90000,30001,2


### Query 5A: For each department ID retrieve the number of employees in the department.

In [16]:
%%sql 

SELECT DEP_ID, count(*) from Employees
    GROUP BY DEP_ID

 * sqlite:///HR.db
Done.


DEP_ID,count(*)
2,3
5,4
7,3


### Query 5B: For each department retrieve the number of employees in the department, and the average employees salary in the department.

In [17]:
%%sql 

SELECT DEP_ID, count(*), AVG(SALARY) from Employees
    GROUP BY DEP_ID

 * sqlite:///HR.db
Done.


DEP_ID,count(*),AVG(SALARY)
2,3,86666.66666666667
5,4,65000.0
7,3,66666.66666666667


### Query 5C: Label the computed columns in the result set of Query 5B as “NUM_EMPLOYEES” and “AVG_SALARY”.

In [18]:
%%sql 

SELECT DEP_ID, count(*) as NUM_EMPLOYEES, AVG(SALARY) as AVG_SALARY from Employees
    GROUP BY DEP_ID

 * sqlite:///HR.db
Done.


DEP_ID,NUM_EMPLOYEES,AVG_SALARY
2,3,86666.66666666667
5,4,65000.0
7,3,66666.66666666667


### Query 5D: In Query 5C order the result set by Average Salary

In [19]:
%%sql 

SELECT DEP_ID, count(*) as NUM_EMPLOYEES, AVG(SALARY) as AVG_SALARY from Employees
    GROUP BY DEP_ID
    ORDER BY AVG_SALARY

 * sqlite:///HR.db
Done.


DEP_ID,NUM_EMPLOYEES,AVG_SALARY
5,4,65000.0
7,3,66666.66666666667
2,3,86666.66666666667


### Query 5E: In Query 5D limit the result to departments with fewer than 4 employees

In [20]:
%%sql 

SELECT DEP_ID, count(*) as NUM_EMPLOYEES, AVG(SALARY) as AVG_SALARY from Employees
    GROUP BY DEP_ID
    HAVING count(*) < 4
    ORDER BY AVG_SALARY

 * sqlite:///HR.db
Done.


DEP_ID,NUM_EMPLOYEES,AVG_SALARY
7,3,66666.66666666667
2,3,86666.66666666667


### BONUS Query 6: Similar to 4B but instead of department ID use department name. Retrieve a list of employees ordered by department name, and within each department ordered alphabetically in descending order by last name.

In [21]:
%%sql

SELECT D.DEP_NAME, E.F_NAME, E.L_NAME from Employees as E, Departments as D
    WHERE E.DEP_ID = D.DEPT_ID_DEP
    ORDER BY D.DEP_NAME, E.L_NAME desc;

 * sqlite:///HR.db
Done.


DEP_NAME,F_NAME,L_NAME
Architect Group,John,Thomas
Architect Group,Ahmed,Hussain
Architect Group,Nancy,Allen
Design Team,Mary,Thomas
Design Team,Andrea,Jones
Design Team,Bharath,Gupta
Software Group,Steve,Wells
Software Group,Steve,Wells
Software Group,Santosh,Kumar
Software Group,Santosh,Kumar


In [22]:
conn.close()